# Predictive Analytics Assignment Group 12
##### Group Members:
| Name | SNR |
| :---- | :--- |
|Nadya Hagen | 2049115 |
| Kenéz Kovács | 2040678 |
| Aryo Bimo Nugroho | 2039696 |
| Sheva Sonia Rahmani | 2075109 |

## The following is a model that aims to predict the direction of the movement of the price of a dogecoin based on Elon Musk's tweets ###

In [52]:
# Loading libraries, getting resources:
import pandas as pd
import nltk
import random
import csv
import random
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.probability import FreqDist

[nltk_data] Downloading package stopwords to /Users/sheva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Data initialization
We expect the user of the model to know their dataset well, the code itself does not have data preparative features. In our case the raw data was processed using mostly pandas and excel features, we felt like the code needn't be  included here as it is not remotely reproducible anyways

In [53]:
import pandas as pd
data = pd.read_csv('/Users/sheva/Downloads/data_fluctuations.csv', delimiter = ',')

### Creating the functions that classify the data

In [54]:
# We create the function that makes a list of lists of words of the tweets 

def into_words(tweetcolumn):
    wordlist = []
    for tweet_sentence in tweetcolumn:
            tweet_sentence_string_lower = str(tweet_sentence).lower()
            tweet_words = tweet_sentence_string_lower.split()
            wordlist.append(tweet_words)
    return(wordlist)

# We create the function that removes stopwords from the list, thus creating a stopword-free list of words

def filterstopwords(wordlist):
    filtered_wordlist = []
    stop_words = list(stopwords.words('english'))
    for lists in wordlist:
        for words in lists:
            if  words.lower() not in stop_words:
                
                filtered_wordlist.append(words.lower())
    return(filtered_wordlist)

# We create the function that makes a list of the top 5000 words used by Elon Musk in his tweetings

def top3kwordizer(filtered_wordlist):
    all_words = nltk.FreqDist(filtered_wordlist)
    all_features = list(all_words)[:4000]
    return all_features

# We create the function that makes the list of lists that contain dictionaries with the features

def list_of_dicts(tweet_sentence, all_features): 
    tweet_words = set(tweet_sentence)
    features = {}
    for word in all_features:
        if word in tweet_words:
            features['contains({})'.format(word)] = True
        else:
            features['contains({})'.format(word)] = False
    return features

## Dataset to classified set of data
Here we use the functions which we have defined above to go from our original dataset to a classified set of data

In [55]:
# Running the functions one by one to classify the data
wordized_list = into_words(data['tweet']) # we have the list of lists of words
filteredwords = filterstopwords(wordized_list) # we have the list of all words filtered free of stopwords
top3kwords = top3kwordizer(filteredwords) # we have the top100 words from the list above
dictslist = list() # we get the list of list of dictionaries containing the features and if theyre in the tweet or not
for tweet in wordized_list: 
        dictslist.append(list_of_dicts(tweet, top3kwords))
        
# Using pandas's very nice dataframe feature to build our classified dataset

workingdataframe = pd.DataFrame()
workingdataframe['features'] = (pd.Series(list(dictslist)))
workingdataframe['direction'] = data['direction']

# Creating the list of lists that the model will train on

finaldata = workingdataframe.values.tolist()

### Training the model
All that is left to do is to train the model. We first create a training data set to train our model upon and a test data set to test the model's accuracy on.

In [56]:
# Training the model

train_set = finaldata[450:]
test_set = finaldata[:450]
classifier = nltk.NaiveBayesClassifier.train(train_set)
acc = nltk.classify.accuracy(classifier, test_set)
print("The accuracy of the model is : ", acc)
classifier.show_most_informative_features()

The accuracy of the model is :  0.5377777777777778
Most Informative Features
     contains(@tashaark) = True                0 : 1      =     11.3 : 1.0
      contains(however,) = True                1 : 0      =      8.4 : 1.0
         contains(tests) = True                0 : 1      =      8.3 : 1.0
            contains(🔥🔥) = True                1 : 0      =      7.6 : 1.0
contains(@justpaulinelol) = True                0 : 1      =      7.0 : 1.0
contains(@teslaratiteam) = True                1 : 0      =      6.9 : 1.0
           contains(pcr) = True                0 : 1      =      6.4 : 1.0
         contains(turns) = True                0 : 1      =      6.4 : 1.0
     contains(@aarons5_) = True                1 : 0      =      6.2 : 1.0
contains(@caspar_stanley) = True                0 : 1      =      5.8 : 1.0


In [60]:
import collections
from nltk.metrics.scores import (precision, recall)

refsets = collections.defaultdict(lambda:train_set)
testsets = collections.defaultdict(lambda:test_set)

for i, (feats, label) in enumerate(test_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    
print( 'Precision:', nltk.metrics.precision(refsets['1'], testsets['1']) )
print( 'Recall:', nltk.metrics.recall(refsets['1'], testsets['1']) )
# `'pos'` is for the "positive" (as opposed to "negative") label 

AttributeError: 'list' object has no attribute 'add'

### Hope you enjoyed using the model, good luck in life!